In [7]:
import csv
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import glob, os
from numpy import savetxt, loadtxt, asarray



tqdm.pandas()

In [13]:
all_nets = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/DataForHASpredictor2/all_nets.json'
more_nets = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/DataForHASpredictor2'

# parsed_nondups_val1 = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_val1_fix.txt'
# parsed_nondups_train1 = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_train1_fix.txt'

# parsed_nondups_val1_csv = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_val1.csv'
# parsed_nondups_train1_csv = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_train1.csv'

# parsed_nondups_val2 = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_val2.txt'
# parsed_nondups_train2 = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_train2.txt'


#ALL
parsed_nondups_val_csv = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/DataForHASpredictor2/new_parsed_val.csv'
parsed_nondups_train_csv = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/DataForHASpredictor2/new_parsed_train.csv'

# parsed_nondups_val_csv = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_val_2.csv'
# parsed_nondups_train_csv = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_train_2.csv'

# Convert to csv

In [14]:
%pycache
number = 2

with open(parsed_nondups_val2, 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open(f'parsed_nondups_val{number}.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerows(lines)

with open(parsed_nondups_train2, 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open(f'parsed_nondups_train{number}.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerows(lines)

# train_hw = pd.read_csv(parsed_nondups_train1_csv)
# val_hw = pd.read_csv(parsed_nondups_val1_csv)




UsageError: Line magic function `%pycache` not found.


# Additional data

In [15]:
#%pycache
os.chdir(more_nets)
add_train = glob.glob("train*.txt")
add_val =  glob.glob("val*.txt")

with open(f'new_parsed_train.csv', 'w') as out_file:
    c=0
    writer = csv.writer(out_file)
    for train in add_train:
        print(train)
        with open(more_nets +'/'+ train, 'r') as in_file:
            # for line in in_file:
            #     print(line[0])
            stripped = (line.strip() for line in in_file)
            #print(list(stripped))
            #print("-----------")
            lines = (line.split(",") for line in stripped if line)
            #print(list(lines))
           # print("---------")
            # if list(lines)[0]=='C':
            #     print(list(lines))
           # writer = csv.writer(out_file)
            writer.writerows(lines)


with open(f'new_parsed_val.csv', 'w') as out_file:
    c=0
    for val in add_val:
        with open(more_nets +'/'+ val, 'r') as in_file:
            stripped = (line.strip() for line in in_file  )
            lines = (line.split(",") for line in stripped if line)
            writer = csv.writer(out_file)
            writer.writerows(lines)



train_00001600-00001649.txt
train_00001850-00001899.txt
train_00002000-00002049.txt
train_00002300-00002349.txt
train_00002200-00002249.txt
train_00001950-00001999.txt
train_00002100-00002149.txt
train_00001700-00001749.txt
train_00001750-00001799.txt
train_00002250-00002299.txt
train_00002150-00002199.txt
train_00001900-00001949.txt
train_00002050-00002099.txt
train_00001800-00001849.txt
train_00002350-00002399.txt
train_00001650-00001699.txt


In [17]:
train_hw = pd.read_csv(parsed_nondups_train_csv)
val_hw = pd.read_csv(parsed_nondups_val_csv)


train_hw = train_hw[train_hw.C != 'C']
val_hw = val_hw[val_hw.C != 'C']


In [18]:
train_hw[2531:2534]

,C,B,name,G,J,mac_num,max_filter_size,mac_array_num,total_power,bw_power,core_power,data_bits,sram_size
2531,748323.0,2679043.0,00001619,8141295.936344964,9.865678969799997e-05,65.0,4098.0,7.0,14.662443587919999,10.716172,3.9462715879199988,2048.0,379904.0
2532,6129302.0,7236639.0,00001619,1698060.5572549982,8.896639770600013e-05,28.0,512.0,2.0,32.50521190824001,28.946556,3.5586559082400053,128.0,80000.0
2533,6015364.0,7236639.0,00001619,1757963.4398911092,8.506332450000004e-05,41.0,512.0,2.0,32.349088980000005,28.946556,3.4025329800000015,128.0,80000.0


# Filter 70% size

In [12]:
%pycache
val_hw["total_power"]=pd.to_numeric(val_hw["total_power"], errors='coerce')
train_hw["total_power"]=pd.to_numeric(train_hw["total_power"], errors='coerce')


train_hw = train_hw[(train_hw.total_power) < 200]
val_hw = val_hw[(val_hw.total_power) < 200 ]


train_hw.to_csv( "parsed_nondups_train_lower_power.csv", index=False, encoding='utf-8-sig')
val_hw.to_csv( "parsed_nondups_val_lower_power.csv", index=False, encoding='utf-8-sig')

UsageError: Line magic function `%pycache` not found.


# Neural Networks Dictionnary

In [21]:


with open(all_nets, 'r') as f:
    nets_dict = json.load(f)


os.chdir(more_nets)
all_filenames = glob.glob("*.json")
for file in all_filenames:
    with open(more_nets +'/'+ file, 'r') as f:
        file_dict = json.load(f)
        nets_dict.update(file_dict)
len(nets_dict.keys())

def number_zeros(k):
    nb = len(k)
    return "0"*(8-nb)


def f(mydict):
    return dict((number_zeros(k)+k,f(v) if hasattr(v,'keys') else v) for k,v in mydict.items())

In [22]:
nets_dict = f(nets_dict)

## Max block size

In [25]:
max_blocks = 0
for k,nn in nets_dict.items():
    if len(nn)>max_blocks:
        max_blocks=len(nn)

max_blocks = max_blocks +1 # add fc layer at the end


In [26]:
max_blocks

37

# General information 

nets_dict_keys :0000 to 00531 (532)

Train : 453 different NN

Val : 50 different NN


# Size dataset

#### Train 1: 137,090
#### Val 1: 13,804
#### Train 2: 14,532
#### Val 2: 2,882

In [27]:
def convert_NN_name(x):
    x=int(x)
    x=str(x)
    while len(str(x))<8 :
        x = '0'+str(x)
    return x

train_hw['name']=train_hw['name'].apply(lambda x : convert_NN_name(x))
val_hw['name']=val_hw['name'].apply(lambda x : convert_NN_name(x))

In [28]:
train_hw.head()

,C,B,name,G,J,mac_num,max_filter_size,mac_array_num,total_power,bw_power,core_power,data_bits,sram_size
0,11944703.0,3771067.0,00001642,1658074.656352509,0.00016476742908900027,20.0,512.0,2.0,21.67496516356001,15.084268,6.5906971635600105,128.0,80000.0
1,8433638.0,4002079.0,00001642,1570522.2101876577,0.00015681514086299977,21.0,512.0,3.0,22.28092163451999,16.008316,6.272605634519991,128.0,60000.0
2,10753855.0,4002079.0,00001642,1411966.0827421746,0.00014640667743900034,30.0,512.0,2.0,21.864583097560015,16.008316,5.856267097560014,128.0,60000.0
3,1110008.0,3708652.0,00001642,10185009.221130842,0.00010648946978700016,189.0,1536.0,14.0,19.094186791480006,14.834608,4.259578791480006,512.0,99968.0
4,11869528.0,4002079.0,00001642,1374615.4980233472,0.00017413813578299998,20.0,1024.0,2.0,22.97384143132,16.008316,6.965525431319999,128.0,60000.0


In [29]:
train_hw[-2:-1]

,C,B,name,G,J,mac_num,max_filter_size,mac_array_num,total_power,bw_power,core_power,data_bits,sram_size
35423,51108099.0,39678749.0,00001695,2166257.3203038117,0.002254929273794983,57.0,1536.0,2.0,248.91216695179932,158.714996,90.19717095179931,512.0,99968.0


# Add neural network to dataframe

In [30]:
train_hw["NN"]=train_hw['name'].map(nets_dict)
val_hw["NN"]=val_hw['name'].map(nets_dict)

In [31]:
train_hw.head()

,C,B,name,G,J,mac_num,max_filter_size,mac_array_num,total_power,bw_power,core_power,data_bits,sram_size,NN
0,11944703.0,3771067.0,00001642,1658074.656352509,0.00016476742908900027,20.0,512.0,2.0,21.67496516356001,15.084268,6.5906971635600105,128.0,80000.0,"[[1, 31, 2, 3, 0, conv], [1, 31, 1, 3, 0, dw],..."
1,8433638.0,4002079.0,00001642,1570522.2101876577,0.00015681514086299977,21.0,512.0,3.0,22.28092163451999,16.008316,6.272605634519991,128.0,60000.0,"[[1, 31, 2, 3, 0, conv], [1, 31, 1, 3, 0, dw],..."
2,10753855.0,4002079.0,00001642,1411966.0827421746,0.00014640667743900034,30.0,512.0,2.0,21.864583097560015,16.008316,5.856267097560014,128.0,60000.0,"[[1, 31, 2, 3, 0, conv], [1, 31, 1, 3, 0, dw],..."
3,1110008.0,3708652.0,00001642,10185009.221130842,0.00010648946978700016,189.0,1536.0,14.0,19.094186791480006,14.834608,4.259578791480006,512.0,99968.0,"[[1, 31, 2, 3, 0, conv], [1, 31, 1, 3, 0, dw],..."
4,11869528.0,4002079.0,00001642,1374615.4980233472,0.00017413813578299998,20.0,1024.0,2.0,22.97384143132,16.008316,6.965525431319999,128.0,60000.0,"[[1, 31, 2, 3, 0, conv], [1, 31, 1, 3, 0, dw],..."


# Treat NN

### Calculate flops

In [32]:
# FIXME DIDNT COUNT ACTIVATION

def conv_flops(hout,cin,cout,k,tensor_in,skip=0):
    flops = 2*k*k*cin*hout*hout*cout
    if skip ==1:
        flops += tensor_in
    return flops

def dw_flops(hout,c,k,tensor_in,skip=0, mult=1):
    flops = 2*k*k*c*hout*hout
    if skip == 1:
        flops +=tensor_in
    return flops

def fcc_flops(i, j):
    return (2*i-1)*j

def calculate_flop(convtype,hin,hout,cin,cout,k,exp,tensor_in, skip=0, mult=1,skip_op=False):
    if skip_op==True:
        return 0
    else:
        if convtype=='conv':
            return conv_flops(hout,cin,cout,k, tensor_in, skip)
        elif convtype=='dw':
            return dw_flops(hout,cin,k,tensor_in,skip)
        elif convtype=='inv':
            hout_pointwise = hin
            return conv_flops(hout=hout_pointwise, cin=cin,cout=cin*exp,k=1, tensor_in=tensor_in, skip=skip) + \
        dw_flops(hout=hout,c=cin*exp,k=k,tensor_in=0,skip=0) + \
        conv_flops(hout=hout,cin=cin*exp,cout=cout,k=1, tensor_in=0, skip=0)


 ### Calculate weights

In [33]:

def conv_weights(cin,cout,k):
    return (k*k*cin+1)*cout

def dw_weights(cin,k,mult=1):
    #dw = equi cin=1 and multiply by cout
    #return conv_flops(hout,1,cin,k)*mult
    return (k*k+1)*cin

def fcc_weight(i, j):
    return
    #return (2*i-1)*j

def calculate_weights(convtype,cin,cout,k,exp,mult=1):
    if convtype=='conv':
        return conv_weights(cin,cout,k)
    elif convtype=='dw':
        return dw_weights(cin,k)
    elif convtype=='inv':
        return conv_weights(cin, cin*exp, 1)+ dw_weights(cin*exp,k,mult) + conv_weights(cin*exp,cout,1)
        

## Distinguish columns between convtypes

In [34]:

def distinct_contypes(NN_frame, columns_names, conv_types):
    columns_names_augmented = [ f'{name}_{conv_type}' for conv_type in conv_types for name in columns_names]
    NN_frame_processed= pd.DataFrame(None, index=np.arange(len(NN)+1), columns=columns_names_augmented)

    ### Len(NN)+1  beceause +1 FC layer at the end
    for conv_type in conv_types:
        columns_conv_type = [f'{columns_name}_{conv_type}' for columns_name in columns_names]
        NN_frame_processed[columns_conv_type] = NN_frame.loc[NN_frame['convtype']==conv_type, columns_names]
    return NN_frame_processed.fillna(0)


# Treat the neural Network

In [35]:

nb_classes= 1000
separate_types = False

# def no_relu(NN_frame):  
#     NN_frame['convtype']= NN_frame['convtype'].apply(lambda x : 'conv' if x=='conv_norelu' else x)

def treat_NN(NN_frame, values_to_keep, separate_types):
    fc_block = {'exp':1, 'c_out':1000, 's':1, 'k':1, 'skip':0,'convtype':'conv'}
    NN_frame=NN_frame.append(fc_block, ignore_index=True)
    import pdb
   # pdb.set_trace()

    #no_relu
    NN_frame['convtype']= NN_frame['convtype'].apply(lambda x : 'conv' if x=='conv_norelu' else x)

    NN_frame['c_in'] = np.roll(NN_frame['c_out'], 1)
    NN_frame['c_in'][0]=3 # RGB channels

    NN_frame['k2']=NN_frame['k'].apply(lambda x : x*x)
    NN_frame['hidden_dim']=NN_frame['exp']*NN_frame['c_in']

    NN_frame['h_in'] = np.nan
    NN_frame['h_in'][0]=224 # Size imagenet - hardcoded
    for i in range(1, len(NN_frame)):
        if NN_frame.loc[i-1, 's']==1:
            NN_frame.loc[i, 'h_in']=NN_frame.loc[i-1, 'h_in']
        elif NN_frame.loc[i-1, 's']==2:
            NN_frame.loc[i, 'h_in']=NN_frame.loc[i-1, 'h_in']/2
        else : 
            raise NameError('Dont know the paddind')


    NN_frame['h_out'] = np.roll(NN_frame['h_in'], -1)
    NN_frame['h_out'][NN_frame.last_valid_index()]=NN_frame['h_in'][NN_frame.last_valid_index()]/NN_frame.loc[NN_frame.last_valid_index(), 's']

    NN_frame['tensor_in']=NN_frame['c_in']*NN_frame['h_in']*NN_frame['h_in']
    NN_frame['tensor_out']=NN_frame['c_out']*NN_frame['h_out']*NN_frame['h_out']

    NN_frame['weights']=NN_frame.apply(lambda x : calculate_weights(x.convtype,x.c_in,x.c_out,x.k,x.exp), axis=1)
    NN_frame['FLOPS']= NN_frame.apply(lambda x : calculate_flop(x.convtype,x.h_in,x.h_out,x.c_in,x.c_out,x.k,x.exp, x.tensor_in,x.skip), axis=1)
    #return  NN_frame
    if separate_types:
        NN_frame=NN_frame.loc[:, values_to_keep + ['convtype']]
        NN_frame =distinct_contypes(NN_frame, values_to_keep, conv_types)
        #values_to_keep = [f'{value}_{type}' for type in conv_types for value in values_to_keep]
    else:
        NN_frame =NN_frame.loc[:, values_to_keep]
    return  NN_frame





## Example NN_treatment

In [36]:
conv_types = ['conv', 'dw', 'inv']
columns_names = ['exp','c_out','s','k','skip']
values_to_keep = ['FLOPS','weights','tensor_in','tensor_out','hidden_dim','k2', 'skip']
#values_to_keep_plus_type = ['FLOPS','weights','tensor_in','tensor_out','hidden_dim','k2', 'skip', 'convtype']



NN = nets_dict['00000001']
NN_frame = pd.DataFrame(NN, columns = columns_names+ ['convtype',])

#distinct =distinct_contypes(NN_frame, columns_names, conv_types)
NN_00001 = treat_NN(NN_frame, values_to_keep, False)
NN_00001
#distinct.fillna(0)

,FLOPS,weights,tensor_in,tensor_out,hidden_dim,k2,skip
0,23708160.0,980,150528.0,439040.0,3,9,0
1,7902720.0,350,439040.0,439040.0,35,9,0
2,29854720.0,1224,439040.0,426496.0,35,1,0
3,181260800.0,11909,426496.0,28224.0,170,25,0
4,7620480.0,1314,28224.0,28224.0,45,9,0
5,3951360.0,1406,28224.0,8624.0,45,9,0
6,2673440.0,1826,8624.0,8624.0,55,9,0
7,2121504.0,3379,8624.0,2548.0,66,25,0
8,2293200.0,6019,2548.0,2548.0,78,49,0
9,2293200.0,6019,2548.0,2548.0,78,49,0


# Verify properperly calculate number of weights

## A) Initialize one NN : nets_dict[00001]

In [37]:
import tensorflow as tf
#import keras
from keras.models import Model
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dropout
from keras.layers import Activation, BatchNormalization, Add, Reshape, DepthwiseConv2D
from keras.utils.vis_utils import plot_model

from keras import backend as K
def relu6(x):
    return K.relu(x, max_value=6.0)

def _conv_block(inputs, filters, kernel, strides):
    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    x = Conv2D(filters, kernel, padding='same', strides=strides)(inputs)
    return x

def _bottleneck(inputs, filters, kernel, t, alpha, s, r=False):
    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    tchannel = K.int_shape(inputs)[channel_axis] * t
    cchannel = int(filters * alpha)
    x = _conv_block(inputs, tchannel, (1, 1), (1, 1))
    x = DepthwiseConv2D(kernel, strides=(s, s), depth_multiplier=1, padding='same')(x)
    x = Conv2D(cchannel, (1, 1), strides=(1, 1), padding='same')(x)
    if r:
        x = Add()([x, inputs])
    return x

def _inverted_residual_block(inputs, filters, kernel, t, alpha, strides, n):
    x=Activation(relu6)(inputs)
    x = _bottleneck(x, filters, kernel, t, alpha, strides)
    for i in range(1, n):
        x = _bottleneck(x, filters, kernel, t, alpha, 1, True)
    x=Activation(relu6)(x)
    return x

def Net1(input_shape, k, alpha=1.0):
    inputs = Input(shape=input_shape)

    x= Conv2D(35, 3, padding='same', strides=(2,2))(inputs)
    x = DepthwiseConv2D((3,3),  activation='relu', padding='same')(x)
    x= Conv2D(34, 1, padding='same', strides=(1,1))(x)

    x = _inverted_residual_block(x, 9, (5, 5), t=5, alpha=alpha, strides=2, n=1)
    x = _inverted_residual_block(x, 9, (3, 3), t=5, alpha=alpha, strides=1, n=1)
    x = _inverted_residual_block(x, 11, (3, 3), t=5, alpha=alpha, strides=2, n=1)
    x = _inverted_residual_block(x, 11, (3, 3), t=5, alpha=alpha, strides=1, n=1)

    x = _inverted_residual_block(x, 13, (5, 5), t=6, alpha=alpha, strides=2, n=1)
    x = _inverted_residual_block(x, 13, (7, 7), t=6, alpha=alpha, strides=1, n=1)
    x = _inverted_residual_block(x, 13, (7, 7), t=6, alpha=alpha, strides=1, n=1)
    x = _inverted_residual_block(x, 13, (3, 3), t=6, alpha=alpha, strides=1, n=1)

    x = _inverted_residual_block(x, 30, (3, 3), t=3, alpha=alpha, strides=2, n=1)
    x = _inverted_residual_block(x, 30, (3, 3), t=3, alpha=alpha, strides=1, n=1)
    x = _inverted_residual_block(x, 29, (3, 3), t=3, alpha=alpha, strides=1, n=1)
    last_filters = 1079

    x = _conv_block(x, last_filters, (1, 1), strides=(1, 1))
    x = GlobalAveragePooling2D()(x)

    x = Reshape((1, 1, last_filters))(x)
    x = Dropout(0.3, name='Dropout')(x)
    #!!!!!!!!!
    x = Conv2D(k, (1, 1), padding='same')(x)
    #!!!!!!!!!
    output = Reshape((k,))(x)
    model = Model(inputs, output)
    # plot_model(model, to_file='images/MobileNetv2.png', show_shapes=True)

    return model


if __name__ == '__main__':
    model_1 = Net1((224, 224, 3), 1000, 1.0)
    print(model_1.summary())

Using TensorFlow backend.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 35)      980       
_________________________________________________________________
depthwise_conv2d_1 (Depthwis (None, 112, 112, 35)      350       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 34)      1224      
_________________________________________________________________
activation_1 (Activation)    (None, 112, 112, 34)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 170)     5950      
_________________________________________________________________
depthwise_conv2d_2 (Depthwis (Non

# Model total param = Total params: 1,164,580

In [38]:
sum(NN_00001['weights'])

1164580

In [39]:
 
##### FIXME!!!!!!!!!!!
def get_flops():
    session = tf.compat.v1.Session()
    graph = tf.compat.v1.get_default_graph()

    with graph.as_default():
        with session.as_default():
            model_2 = Net1((224, 224, 3), 1000, 1.0)
            run_meta = tf.compat.v1.RunMetadata()
            opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
            # Optional: save printed results to file
            # flops_log_path = os.path.join(tempfile.gettempdir(), 'tf_flops_log.txt')
            # opts['output'] = 'file:outfile={}'.format(flops_log_path)

            # We use the Keras session graph in the call to the profiler.
            flops = tf.compat.v1.profiler.profile(graph=graph,
                                                  run_meta=run_meta, cmd='op', options=opts)

            return flops.total_float_ops
get_flops()

#print(sum(NN_00001['FLOPS']))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


2321134

In [40]:
columns_names

['exp', 'c_out', 's', 'k', 'skip']

In [41]:
conv_types

['conv', 'dw', 'inv']

In [42]:
values_to_keep

['FLOPS', 'weights', 'tensor_in', 'tensor_out', 'hidden_dim', 'k2', 'skip']

# Get the final representation of the NN 

In [43]:

def no_relu(NN_frame):  
    NN_frame['convtype']= NN_frame['convtype'].apply(lambda x : 'conv' if x=='conv_norelu' else x)


def NN_to_dataframe(NN_list, columns_names, conv_types, values_to_keep,separate_types):
    NN_frame = pd.DataFrame(NN_list, columns = columns_names+ ['convtype',])
    no_relu(NN_frame)
    return treat_NN(NN_frame, values_to_keep,separate_types)

    #NN_distinct_contypes = distinct_contypes(NN_frame, columns_names, conv_types)
    #return NN_frame
    #return NN_distinct_contypes


NN_df = NN_to_dataframe(NN, columns_names, conv_types,values_to_keep,separate_types)
NN_df

,FLOPS,weights,tensor_in,tensor_out,hidden_dim,k2,skip
0,23708160.0,980,150528.0,439040.0,3,9,0
1,7902720.0,350,439040.0,439040.0,35,9,0
2,29854720.0,1224,439040.0,426496.0,35,1,0
3,181260800.0,11909,426496.0,28224.0,170,25,0
4,7620480.0,1314,28224.0,28224.0,45,9,0
5,3951360.0,1406,28224.0,8624.0,45,9,0
6,2673440.0,1826,8624.0,8624.0,55,9,0
7,2121504.0,3379,8624.0,2548.0,66,25,0
8,2293200.0,6019,2548.0,2548.0,78,49,0
9,2293200.0,6019,2548.0,2548.0,78,49,0


In [44]:
NN

[[1, 35, 2, 3, 0, 'conv'],
 [1, 35, 1, 3, 0, 'dw'],
 [1, 34, 1, 1, 0, 'conv_norelu'],
 [5, 9, 2, 5, 0, 'inv'],
 [5, 9, 1, 3, 0, 'inv'],
 [5, 11, 2, 3, 0, 'inv'],
 [5, 11, 1, 3, 0, 'inv'],
 [6, 13, 2, 5, 0, 'inv'],
 [6, 13, 1, 7, 0, 'inv'],
 [6, 13, 1, 7, 0, 'inv'],
 [6, 13, 1, 3, 0, 'inv'],
 [3, 30, 2, 3, 0, 'inv'],
 [3, 30, 1, 3, 1, 'inv'],
 [3, 29, 1, 3, 0, 'inv'],
 [1, 1079, 1, 1, 0, 'conv']]

## Add HW params to NN_dataframes

In [45]:
params = ['mac_num','mac_array_num','data_bits','sram_size','max_filter_size']

def add_HW_param_column(df,mac_num, mac_array_num, data_bits, sram_size, max_filter_size):
    df['mac_num']=mac_num
    df['mac_array_num']=mac_array_num
    df['total_macs']=  mac_num * mac_array_num
    df['data_bits']=data_bits
    df['sram_size']=sram_size
    df['max_filter_size']=max_filter_size


def add_HW_param_line(NN_frame, mac_num, mac_array_num, data_bits, sram_size, max_filter_size):
    HW_param_line =pd.DataFrame([[mac_num]*(NN_frame.shape[1])], columns=NN_frame.columns)
    HW_param_line = HW_param_line.append(pd.DataFrame([[mac_array_num]*(NN_frame.shape[1])], columns=NN_frame.columns))
    HW_param_line = HW_param_line.append(pd.DataFrame([[mac_num*mac_array_num]*(NN_frame.shape[1])], columns=NN_frame.columns))
    HW_param_line =HW_param_line.append(pd.DataFrame([[data_bits]*(NN_frame.shape[1])], columns=NN_frame.columns))
    HW_param_line =HW_param_line.append(pd.DataFrame([[sram_size]*(NN_frame.shape[1])], columns=NN_frame.columns))
    HW_param_line =HW_param_line.append(pd.DataFrame([[max_filter_size]*(NN_frame.shape[1])], columns=NN_frame.columns))
    return HW_param_line.append(NN_frame)


#not_used
def extend_to_max_blocks(NN_frame, value):
    for i in range (max_blocks-len(NN_frame)):
        NN_frame=NN_frame.append(pd.DataFrame([[0]*(NN_frame.shape[1])], index=[i+len(NN_frame),] ,columns=NN_frame.columns))
    return NN_frame

#not_used
def add_zero_blocks(arr):
    zero_blocks = np.zeros(( max_blocks-arr.shape[0],arr.shape[1]))
    return np.append(arr, zero_blocks, axis=0)



In [61]:
%pycache
train_hw= train_hw[:3]
val_hw = val_hw[:3]

UsageError: Line magic function `%pycache` not found.


# Apply and save

In [46]:
#%pycache
values_to_keep = ['FLOPS','weights','tensor_in','tensor_out','hidden_dim','k2', 'skip']
separate_types = False
# if separate_types:
#     values_to_keep = [f'{value}_{type}' for type in conv_types for value in values_to_keep]

name = 'from_1600'


# Convert NN to dataframe with values of interest
train_hw["NN_dataframe"] = train_hw["NN"].progress_apply(lambda x : NN_to_dataframe(x,columns_names, conv_types, values_to_keep,separate_types))
#train_hw.progress_apply(lambda x : add_HW_param_column(x.NN_dataframe,x.mac_num,x.mac_array_num, x.data_bits, x.sram_size, x.max_filter_size), axis=1)
train_hw['NN_dataframe']=train_hw['NN_dataframe'].progress_apply(lambda x : np.array(x))
#complete to zero#complete to zero
#train_hw['NN_dataframe']= train['NN_dataframe'].progress_apply(lambda x : add_zero_blocks(x))#complete to zero#complete to zero
train_hw['NN_dataframe'].to_csv(f'train_{name}.csv')


#####################################

val_hw["NN_dataframe"] = val_hw["NN"].progress_apply(lambda x : NN_to_dataframe(x,columns_names, conv_types, values_to_keep,separate_types))
#val_hw.progress_apply(lambda x : add_HW_param_column(x.NN_dataframe,x.mac_num,x.mac_array_num, x.data_bits, x.sram_size, x.max_filter_size), axis=1)
val_hw['NN_dataframe']=val_hw['NN_dataframe'].progress_apply(lambda x : np.array(x))

#complete to zero#complete to zero
#val['NN_dataframe']= val['NN_dataframe'].progress_apply(lambda x : add_zero_blocks(x))#complete to zero#complete to zero
val_hw.to_csv(f'val_{name}.csv')


100%|██████████| 4451/4451 [00:00<00:00, 14492.38it/s]


In [35]:
train_hw

,mac_num,mac_array_num,data_bits,sram_size,max_filter_size,G,C,B,J,name,bw_power,core_power,total_power,NN,NN_dataframe
0,118.0,2.0,256.0,179968.0,2048.0,3.636011e+06,18770712.0,19987780.0,0.000533,00000,79.951120,21.330414,101.281534,"[[1, 9, 2, 3, 0, conv], [1, 9, 1, 3, 0, dw], [...","[[6096384.0, 252.0, 150528.0, 112896.0, 3.0, 9..."
1,87.0,2.0,512.0,99968.0,2048.0,2.325259e+06,14362735.0,21849702.0,0.000570,00000,87.398808,22.816285,110.215093,"[[1, 9, 2, 3, 0, conv], [1, 9, 1, 3, 0, dw], [...","[[6096384.0, 252.0, 150528.0, 112896.0, 3.0, 9..."
2,124.0,2.0,512.0,99968.0,3072.0,2.533459e+06,12739050.0,21849702.0,0.000543,00000,87.398808,21.716202,109.115010,"[[1, 9, 2, 3, 0, conv], [1, 9, 1, 3, 0, dw], [...","[[6096384.0, 252.0, 150528.0, 112896.0, 3.0, 9..."


In [51]:
#%pycache
import ast


path_processed_val_nn="/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/DataForHASpredictor/val_low_power.csv"

def from_np_array(array_string):
    array_string = ','.join(array_string.replace('[ ', '[').split())
    return np.asarray(ast.literal_eval(array_string))

val = pd.read_csv(path_processed_val_nn,converters={'NN_dataframe': from_np_array})

X_train_nn =np.array(val['NN_dataframe'].tolist())
#train = pd.read_csv(path_processed_train_nn,converters={'NN_dataframe': from_np_array})

In [53]:
pd.DataFrame(val['NN_dataframe'][0])

,0,1,2,3,4,5,6
0,16257024.0,672.0,150528.0,301056.0,3.0,9.0,0.0
1,5419008.0,240.0,301056.0,301056.0,24.0,9.0,0.0
2,21676032.0,900.0,301056.0,451584.0,24.0,1.0,0.0
3,158054400.0,9958.0,451584.0,68992.0,144.0,9.0,0.0
4,29252608.0,4862.0,68992.0,68992.0,88.0,9.0,0.0
5,94657024.0,15427.0,68992.0,84672.0,154.0,49.0,0.0
6,80269056.0,13149.0,84672.0,84672.0,162.0,25.0,0.0
7,42674688.0,7047.0,84672.0,84672.0,108.0,9.0,0.0
8,53597376.0,8775.0,84672.0,84672.0,108.0,25.0,1.0
9,104654592.0,17037.0,84672.0,84672.0,162.0,49.0,0.0


In [39]:
pd.DataFrame(X_train_nn)

ValueError: Must pass 2-d input

In [40]:
train_hw = pd.read_csv('/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/DataForHASpredictor/new_train.csv')
train_hw_old = pd.read_csv('/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_train_all.csv')
train_all = pd.read_csv('/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_train_new.csv')

FileNotFoundError: [Errno 2] File /Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_train_all.csv does not exist: '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_train_all.csv'

In [41]:
train_hw = train_hw[train_hw.C != 'C']
#al_hw = val_hw[val_hw.C != 'C']

train_hw.astype(float)

,C,B,name,G,J,mac_num,max_filter_size,mac_array_num,total_power,bw_power,core_power,data_bits,sram_size
0,7268254.0,17970336.0,1501.0,3.598255e+06,0.000879,76.0,512.0,4.0,107.060580,71.881344,35.179236,2048.0,139904.0
1,14613399.0,19142521.0,1501.0,2.277913e+06,0.001274,58.0,2048.0,2.0,127.519998,76.570084,50.949914,2048.0,99968.0
2,22923332.0,18360833.0,1501.0,2.797106e+06,0.000662,118.0,4098.0,2.0,99.933890,73.443332,26.490558,256.0,119936.0
3,25836964.0,20670026.0,1501.0,1.858189e+06,0.001977,29.0,3072.0,2.0,161.743410,82.680104,79.063306,2048.0,80000.0
4,22793145.0,18360833.0,1501.0,2.839804e+06,0.000660,134.0,2048.0,2.0,99.855414,73.443332,26.412082,256.0,119936.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28419,2463773.0,9007084.0,1569.0,2.096227e+07,0.000746,130.0,2048.0,32.0,65.876460,36.028336,29.848124,512.0,379904.0
28420,25364967.0,31532640.0,1569.0,3.160583e+06,0.000921,200.0,1024.0,2.0,162.960370,126.130560,36.829810,256.0,119936.0
28421,4062796.0,5803814.0,1569.0,8.571541e+06,0.000820,108.0,1024.0,9.0,56.009309,23.215256,32.794053,1024.0,320000.0
28422,26034959.0,24410683.0,1569.0,2.564763e+06,0.001388,135.0,2048.0,2.0,153.152159,97.642732,55.509427,512.0,99968.0


In [42]:
train_hw_old.head()

NameError: name 'train_hw_old' is not defined

In [43]:
train_hw.head()

,C,B,name,G,J,mac_num,max_filter_size,mac_array_num,total_power,bw_power,core_power,data_bits,sram_size
0,7268254.0,17970336.0,00001501,3598255.3109029434,0.000879480903465003,76.0,512.0,4.0,107.06058013860012,71.881344,35.17923613860012,2048.0,139904.0
1,14613399.0,19142521.0,00001501,2277912.6023773067,0.0012737478559800043,58.0,2048.0,2.0,127.51999823920016,76.570084,50.949914239200176,2048.0,99968.0
2,22923332.0,18360833.0,00001501,2797106.426879716,0.0006622639444620016,118.0,4098.0,2.0,99.93388977848007,73.443332,26.490557778480063,256.0,119936.0
3,25836964.0,20670026.0,00001501,1858189.091508767,0.0019765826388869756,29.0,3072.0,2.0,161.74340955547902,82.680104,79.06330555547902,2048.0,80000.0
4,22793145.0,18360833.0,00001501,2839804.299988273,0.0006603020464379995,134.0,2048.0,2.0,99.85541385751998,73.443332,26.41208185751998,256.0,119936.0


In [44]:
type(X_train_hw_param['mac_num'][0])

NameError: name 'X_train_hw_param' is not defined

In [45]:
type(X_train_hw_param['total_power'][0])

NameError: name 'X_train_hw_param' is not defined

In [46]:
#train_hw['mac_num']*train_hw['mac_array_num']

X_train_hw_param = train_hw

for i in range(len(X_train_hw_param)):

    if type(X_train_hw_param['mac_num'][i]) != float :
        print (type(X_train_hw_param['mac_num'][i]))
        print(i)
        break

<class 'str'>
0


## Combine CSV files

In [50]:

import os
import glob
import pandas
import pandas as pd
os.chdir("/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data")
extension = 'csv'

trains = ["/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/data_180035train/parsed_nondups_train_2.csv", "/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/DataForHASpredictor2/new_parsed_train.csv"]

vals = ["/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/data_180035train/parsed_nondups_val_2.csv", "/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/DataForHASpredictor2/new_parsed_val.csv"]
#all_filenames = [i for i in glob.glob('*.{}'.format(extension))]


#combine all files in the list
#train_csv = pd.concat([pd.read_csv(f) for f in trains])
#val_csv = pd.concat([pd.read_csv(f) for f in vals])

train_l = []
for f in trains:
    panda = pd.read_csv(f)
    panda = panda[panda.C != 'C']
    panda = panda.astype(float)
    train_l.append(panda)
train_csv = pd.concat(train_l)



val_l = []
for f in vals:
    panda = pd.read_csv(f)
    panda = panda[panda.C != 'C']
    panda = panda.astype(float)
    val_l.append(panda)
val_csv = pd.concat(val_l)



#train_hw = train_hw[train_hw.C != 'C']
#export to csv
train_csv.to_csv( "parsed_nondups_train_3.csv", index=False, encoding='utf-8-sig')
val_csv.to_csv( "parsed_nondups_val_3.csv", index=False, encoding='utf-8-sig')

In [48]:
!ls

all_nets.json              train_7.csv
parsed_nondups_train_1.csv train_7_new.csv
parsed_nondups_train_2.csv txt
parsed_nondups_val_1.csv   val_13_all.csv
parsed_nondups_val_2.csv   val_21_all.csv
train_13_all.csv           val_7.csv
train_21_all.csv           val_7_new.csv


In [49]:
train_hw = pd.read_csv('/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_train_new.csv')

FileNotFoundError: [Errno 2] File /Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_train_new.csv does not exist: '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_train_new.csv'

In [50]:
len(train_hw)

28413